# Alias Generator Functions

When working with REST APIs, using camel case for deserializing/serializing data, and snake case for field names is extremely common.
However, doing this for hundreds of fields and dozens of models is tedious and error prone.

Converting from snake case to camel case is rather systematic, and we could write a function to do this.

In fact, there are 3rd party libraries that provide this very capability.

Pydantic also does.

These case conversion functions are documented [here](https://docs.pydantic.dev/latest/api/config/#pydantic.alias_generators)

Let's look at them:

In [66]:
from pydantic.alias_generators import to_camel, to_snake, to_pascal
print(to_camel("last_name"))
print(to_snake("lastName"))
print(to_pascal("last_naMe"))

lastName
last_name
LastName


The reason we have those functions is that we can configure our **model** to auto generate field aliases using one of those functions.
But any function that converts one string (the field name) into another (an alias) can also be used.

In [67]:
def make_upper(in_str: str) -> str:
    return in_str.upper()


make_upper("last_name")

'LAST_NAME'

Now let's attach this function to our model definition, using, of course, `model_config`:

In [68]:
from pydantic import BaseModel, ConfigDict, Field, ValidationError


class Person(BaseModel):
    model_config = ConfigDict(alias_generator=make_upper)

    id_: int
    first_name: str | None = None
    last_name: str
    age: int | None = None


p = Person(ID_=1, LAST_NAME="Fourier", AGE=62)

print(p.model_dump())
print(p.model_dump(by_alias=True))

{'id_': 1, 'first_name': None, 'last_name': 'Fourier', 'age': 62}
{'ID_': 1, 'FIRST_NAME': None, 'LAST_NAME': 'Fourier', 'AGE': 62}


You'll notice how `id_`'s alias became `ID_` - maybe we don't want that, in which case we can override it:

In [69]:
import pprint


class Person(BaseModel):
    model_config = ConfigDict(alias_generator=make_upper)

    id_: int = Field(alias="ID")
    first_name: str | None = None
    last_name: str
    age: int | None = None


pprint.pprint(Person.model_fields)

{'age': FieldInfo(annotation=Union[int, NoneType], required=False, alias='AGE'),
 'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, alias='FIRST_NAME'),
 'id_': FieldInfo(annotation=int, required=True, alias='ID', alias_priority=2),
 'last_name': FieldInfo(annotation=str, required=True, alias='LAST_NAME')}


In [70]:
p = Person(ID=1, LAST_NAME="Fourier", AGE=62)
print(p)
print(p.model_dump())
print(p.model_dump(by_alias=True))

id_=1 first_name=None last_name='Fourier' age=62
{'id_': 1, 'first_name': None, 'last_name': 'Fourier', 'age': 62}
{'ID': 1, 'FIRST_NAME': None, 'LAST_NAME': 'Fourier', 'AGE': 62}


It is customary, when the data we are deserializing contains Python reserved words, to simply define the field name as that name with an underscore (`_`) appended to the field name, e.g.:
- `id` -> `id_`
- `list` -> `list_`
- `filter` -> `filter_`

We could certainly use the method we just saw, setting an alias generator, and overriding these special cases one by one:

In [71]:
class Model(BaseModel):
    model_config = ConfigDict(alias_generator=to_camel)

    id_: int = Field(alias="id")
    list_: list[str] = Field(alias="list")
    filter_: dict = Field(alias="filter")
    number_elements: list[int]

When we do this we have the following aliases:

In [72]:
Model.model_fields

{'id_': FieldInfo(annotation=int, required=True, alias='id', alias_priority=2),
 'list_': FieldInfo(annotation=list[str], required=True, alias='list', alias_priority=2),
 'filter_': FieldInfo(annotation=dict, required=True, alias='filter', alias_priority=2),
 'number_elements': FieldInfo(annotation=list[int], required=True, alias='numberElements')}